In [16]:
#import required libraries
from bs4 import BeautifulSoup
import lxml
import requests
import pandas as pd
from nltk.corpus import stopwords
from nltk import word_tokenize
%matplotlib inline

In [17]:
#checking avaliable sites for scraping and getting response if HTTP response code permits 

url_list= [
'https://www.simplyscripts.com',
'http://www.script-o-rama.com',
'https://imsdb.com',
'https://www.scriptslug.com'
]

def get_url_status_preview(url):
    response = requests.get(url)
    try:
        print('Checking site status code for:',str(url))

        print('Response recieved:',response.status_code)
        if response.status_code != 200:
            print("Site Unable to Authorize Request")
        else:
            soup = BeautifulSoup(response.content, 'lxml')
            return soup 
    except: 
        print('Error: Check formatting of URL inputs/response request')
        
#create empty list and append with parsed html        
url_info = []
for url in url_list:
  url_info.append(get_url_status(url))
print(url_info)

Checking site status code for: https://www.simplyscripts.com
Response recieved: 200
Checking site status code for: http://www.script-o-rama.com
Response recieved: 200
Checking site status code for: https://imsdb.com
Response recieved: 200
Checking site status code for: https://www.scriptslug.com
Response recieved: 200
[<html><head><title>You are being redirected...</title>
<noscript>Javascript is required. Please enable javascript before you are allowed to see this page.</noscript>
<script>var s={},u,c,U,r,i,l=0,a,e=eval,w=String.fromCharCode,sucuri_cloudproxy_js='',S='eD0nMG1TYScuc3Vic3RyKDMsIDEpICsgJycgKycnKyJlIi5zbGljZSgwLDEpICsgIjVzdWN1ciIuY2hhckF0KDApKyIzbCIuY2hhckF0KDApICsgICcnICsgCiJjc3VjdXIiLmNoYXJBdCgwKSsnYycgKyAgICcnICsnMjEnLnNsaWNlKDEsMikrJzUnICsgICIwIiArICIiICsiNHN1Ii5zbGljZSgwLDEpICsgImFzdWN1ciIuY2hhckF0KDApKyIiICsndUdiJy5jaGFyQXQoMikrJz0zJy5zbGljZSgxLDIpK1N0cmluZy5mcm9tQ2hhckNvZGUoMHgzNSkgKyAnMScgKyAgJzcnICsgICAnJyArJzInICsgICI4ayIuY2hhckF0KDApICsgICcnICsgCiJjdyIuY2hhckF0

In [35]:
#obtaining scriptslug urls from home page to find oscar based categories of interest

script_slug_url = 'https://www.scriptslug.com/'

def get_script_slug_url(url_list):
    response = requests.get(url_list)
    try:
        print('Checking site status code for:',str(url_list))

        print('Response recieved:',response.status_code)
        if response.status_code != 200:
            print("Site Unable to Authorize Request")
        else:
            soup = BeautifulSoup(response.content, 'lxml')
            url_list= [a.get('href') for a in soup.find_all('a', href=True)]
            return url_list
    except: 
        print('Error: Check formatting of URL inputs/response request')

get_script_slug_url(script_slug_url)

Checking site status code for: https://www.scriptslug.com/
Response recieved: 200


['https://www.scriptslug.com/',
 'https://www.scriptslug.com/about',
 'https://www.scriptslug.com/contact',
 'https://www.facebook.com/ScriptSlug/',
 'https://twitter.com/ScriptSlug',
 'https://www.scriptslug.com/scripts/medium/film',
 'https://www.scriptslug.com/scripts/medium/series',
 'https://www.scriptslug.com/scripts/medium/podcast',
 'https://www.scriptslug.com/scripts/genre/action',
 'https://www.scriptslug.com/scripts/genre/adventure',
 'https://www.scriptslug.com/scripts/genre/animation',
 'https://www.scriptslug.com/scripts/genre/biography',
 'https://www.scriptslug.com/scripts/genre/comedy',
 'https://www.scriptslug.com/scripts/genre/crime',
 'https://www.scriptslug.com/scripts/genre/drama',
 'https://www.scriptslug.com/scripts/genre/family',
 'https://www.scriptslug.com/scripts/genre/fantasy',
 'https://www.scriptslug.com/scripts/genre/film-noir',
 'https://www.scriptslug.com/scripts/genre/history',
 'https://www.scriptslug.com/scripts/genre/holiday',
 'https://www.scripts

In [40]:
#similarily for the oscar-based urls we obtain the list of movie urls 

script_slug_oscar_url = ['https://www.scriptslug.com/feature/oscar-nominated-scripts-2022',
  'https://www.scriptslug.com/feature/oscar-nominated-scripts-2021',
  'https://www.scriptslug.com/feature/oscar-nominated-scripts-2020',
  'https://www.scriptslug.com/feature/oscar-nominated-scripts-2019',
  'https://www.scriptslug.com/feature/oscar-nominated-scripts-2018',
  'https://www.scriptslug.com/feature/oscar-nominated-scripts-2017']

#create empty list and append with parsed html        
script_slug_oscar_url_list= []
for url in script_slug_oscar_url:
  script_slug_oscar_url_list.append(get_script_slug_url(url))
print(script_slug_oscar_url_list)

Checking site status code for: https://www.scriptslug.com/feature/oscar-nominated-scripts-2022
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/feature/oscar-nominated-scripts-2021
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/feature/oscar-nominated-scripts-2020
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/feature/oscar-nominated-scripts-2019
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/feature/oscar-nominated-scripts-2018
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/feature/oscar-nominated-scripts-2017
Response recieved: 200
[['https://www.scriptslug.com/', 'https://www.scriptslug.com/about', 'https://www.scriptslug.com/contact', 'https://www.facebook.com/ScriptSlug/', 'https://twitter.com/ScriptSlug', 'https://www.scriptslug.com/feature/oscar-nominated-scripts-2022', 'https://www.scriptslug.com/script/belfast-2

In [47]:
#flatten list of list,deduplicate,and keep only script related urls
from functools import reduce
import operator

script_slug_oscar_url_list_final = set(reduce(operator.concat, script_slug_oscar_url_list))
script_slug_oscar_url_list_final = [x for x in script_slug_oscar_url_list_final if x.startswith('https://www.scriptslug.com/script/')]
script_slug_oscar_url_list_final 

['https://www.scriptslug.com/script/can-you-ever-forgive-me-2018',
 'https://www.scriptslug.com/script/the-power-of-the-dog-2021',
 'https://www.scriptslug.com/script/the-two-popes-2019',
 'https://www.scriptslug.com/script/the-ballad-of-buster-scruggs-2018',
 'https://www.scriptslug.com/script/the-father-2020',
 'https://www.scriptslug.com/script/little-women-2019',
 'https://www.scriptslug.com/script/the-irishman-2019',
 'https://www.scriptslug.com/script/the-favourite-2018',
 'https://www.scriptslug.com/script/first-reformed-2018',
 'https://www.scriptslug.com/script/lady-bird-2017',
 'https://www.scriptslug.com/script/lion-2016',
 'https://www.scriptslug.com/script/jojo-rabbit-2019',
 'https://www.scriptslug.com/script/moonlight-2016',
 'https://www.scriptslug.com/script/belfast-2021',
 'https://www.scriptslug.com/script/nomadland-2020',
 'https://www.scriptslug.com/script/roma-2018',
 'https://www.scriptslug.com/script/joker-2019',
 'https://www.scriptslug.com/script/dune-2021',
 

In [89]:
#Obtaining the .pdf urls of the scripts
import re

def get_script_slug_pdf(url_list):
    response = requests.get(url_list)
    try:
        print('Checking site status code for:',str(url_list))

        print('Response recieved:',response.status_code)
        if response.status_code != 200:
            print("Site Unable to Authorize Request")
        else:
            soup = BeautifulSoup(response.content, 'lxml')
            pdf_list= [a.get('href') for a in soup.find_all('a', href=re.compile(r'(.pdf)'))] 
            return pdf_list
    except: 
        print('Error: Check formatting of URL inputs/response request')


#create empty list and append with parsed html        
script_slug_oscar_pdf_list= []
for url in script_slug_oscar_url_list_final:
  script_slug_oscar_pdf_list.append(get_script_slug_pdf(url))
print(script_slug_oscar_pdf_list)

Checking site status code for: https://www.scriptslug.com/script/can-you-ever-forgive-me-2018
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/script/the-power-of-the-dog-2021
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/script/the-two-popes-2019
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/script/the-ballad-of-buster-scruggs-2018
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/script/the-father-2020
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/script/little-women-2019
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/script/the-irishman-2019
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/script/the-favourite-2018
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/script/first-reformed-2018
Response recieved: 200
Checking 

In [90]:
#generate single list from list-of-lists and then deduplicate urls
script_slug_oscar_pdf_list = (reduce(operator.concat, script_slug_oscar_pdf_list))
script_slug_oscar_pdf_list = (set(script_slug_oscar_pdf_list))
print('The size of Oscar document list will be',len(script_slug_oscar_pdf_list),'screenplays shown below.')
print(script_slug_oscar_pdf_list)

The size of Oscar document list will be 55 screenplays shown below.
{'https://www.scriptslug.com/assets/scripts/hidden-figures-2016.pdf', 'https://www.scriptslug.com/assets/scripts/belfast-2021.pdf', 'https://www.scriptslug.com/assets/scripts/king-richard-2021.pdf', 'https://www.scriptslug.com/assets/scripts/the-two-popes-2019.pdf', 'https://www.scriptslug.com/assets/scripts/a-star-is-born-2018.pdf', 'https://www.scriptslug.com/assets/scripts/nomadland-2020.pdf', 'https://www.scriptslug.com/assets/scripts/the-shape-of-water-2017.pdf', 'https://www.scriptslug.com/assets/scripts/the-ballad-of-buster-scruggs-2018.pdf', 'https://www.scriptslug.com/assets/scripts/promising-young-woman-2020.pdf', 'https://www.scriptslug.com/assets/scripts/the-lobster-2015.pdf', 'https://www.scriptslug.com/assets/scripts/little-women-2019.pdf', 'https://www.scriptslug.com/assets/scripts/the-disaster-artist-2017.pdf', 'https://www.scriptslug.com/assets/scripts/marriage-story-2019.pdf', 'https://www.scriptslug.

In [80]:
#downloading the pdfs
import os
oscar_output_dir = 'C:/Users/U6068968/Downloads/Assessment3/oscar_pdf'

for url in script_slug_oscar_pdf_list:
    response = requests.get(url)
    if response.status_code == 200:
        file_path = os.path.join(oscar_output_dir, os.path.basename(url))
        with open(file_path, 'wb') as f:
            f.write(response.content)

In [91]:
#repeating the above for unremarkable screenplays

script_slug_unremarkable_url = ['https://www.scriptslug.com/scripts/genre/action',
 'https://www.scriptslug.com/scripts/genre/adventure',
 'https://www.scriptslug.com/scripts/genre/animation',
 'https://www.scriptslug.com/scripts/genre/biography',
 'https://www.scriptslug.com/scripts/genre/comedy',
 'https://www.scriptslug.com/scripts/genre/crime',
 'https://www.scriptslug.com/scripts/genre/drama',
 'https://www.scriptslug.com/scripts/genre/family',
 'https://www.scriptslug.com/scripts/genre/fantasy',
 'https://www.scriptslug.com/scripts/genre/film-noir',
 'https://www.scriptslug.com/scripts/genre/history',
 'https://www.scriptslug.com/scripts/genre/holiday',
 'https://www.scriptslug.com/scripts/genre/horror',
 'https://www.scriptslug.com/scripts/genre/music',
 'https://www.scriptslug.com/scripts/genre/musical',
 'https://www.scriptslug.com/scripts/genre/mystery',
 'https://www.scriptslug.com/scripts/genre/romance',
 'https://www.scriptslug.com/scripts/genre/science-fiction',
 'https://www.scriptslug.com/scripts/genre/short',
 'https://www.scriptslug.com/scripts/genre/sports',
 'https://www.scriptslug.com/scripts/genre/superhero',
 'https://www.scriptslug.com/scripts/genre/thriller',
 'https://www.scriptslug.com/scripts/genre/war',
 'https://www.scriptslug.com/scripts/genre/western']

#create empty list and append with parsed html        
script_slug_unremarkable_url_list= []
for url in script_slug_unremarkable_url:
  script_slug_unremarkable_url_list.append(get_script_slug_url(url))
print(script_slug_unremarkable_url_list)

Checking site status code for: https://www.scriptslug.com/scripts/genre/action
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/scripts/genre/adventure
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/scripts/genre/animation
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/scripts/genre/biography
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/scripts/genre/comedy
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/scripts/genre/crime
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/scripts/genre/drama
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/scripts/genre/family
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/scripts/genre/fantasy
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/scripts/genre/fi

In [92]:
script_slug_unremarkable_url_list_final = set(reduce(operator.concat, script_slug_unremarkable_url_list))
script_slug_unremarkable_url_list_final = [x for x in script_slug_unremarkable_url_list_final if x.startswith('https://www.scriptslug.com/script/')]
script_slug_unremarkable_url_list_final

['https://www.scriptslug.com/script/ghost-rider-2007',
 'https://www.scriptslug.com/script/can-you-ever-forgive-me-2018',
 'https://www.scriptslug.com/script/avp-alien-vs-predator-2004',
 'https://www.scriptslug.com/script/all-the-presidents-men-1976',
 'https://www.scriptslug.com/script/crimson-peak-2015',
 'https://www.scriptslug.com/script/about-alex-2014',
 'https://www.scriptslug.com/script/black-christmas-1974',
 'https://www.scriptslug.com/script/ad-astra-2019',
 'https://www.scriptslug.com/script/planes-trains-automobiles-1987',
 'https://www.scriptslug.com/script/coraline-2009',
 'https://www.scriptslug.com/script/american-pie-1999',
 'https://www.scriptslug.com/script/harry-potter-and-the-sorcerers-stone-2001',
 'https://www.scriptslug.com/script/the-afterparty-101-aniq-2022',
 'https://www.scriptslug.com/script/butch-cassidy-and-the-sundance-kid-1968',
 'https://www.scriptslug.com/script/the-boxtrolls-2014',
 'https://www.scriptslug.com/script/black-sails-101-i-2014',
 'http

In [93]:
len(script_slug_unremarkable_url_list_final)

493

In [94]:
import random
random.shuffle(script_slug_unremarkable_url_list_final)

In [96]:
script_slug_unremarkable_url_list_final = script_slug_unremarkable_url_list_final[:55]


['https://www.scriptslug.com/script/bridge-of-spies-2015',
 'https://www.scriptslug.com/script/chernobyl-104-the-happiness-of-all-mankind-2019',
 'https://www.scriptslug.com/script/annabelle-2014',
 'https://www.scriptslug.com/script/d2-the-mighty-ducks-1994',
 'https://www.scriptslug.com/script/isle-of-dogs-2018',
 'https://www.scriptslug.com/script/the-big-short-2015',
 'https://www.scriptslug.com/script/eye-in-the-sky-2016',
 'https://www.scriptslug.com/script/bean-1997',
 'https://www.scriptslug.com/script/childs-play-1988',
 'https://www.scriptslug.com/script/cobra-kai-310-december-19-2021',
 'https://www.scriptslug.com/script/bad-lieutenant-1992',
 'https://www.scriptslug.com/script/blood-diamond-2006',
 'https://www.scriptslug.com/script/how-the-grinch-stole-christmas-2000',
 'https://www.scriptslug.com/script/the-bonfire-of-the-vanities-1990',
 'https://www.scriptslug.com/script/all-about-eve-1950',
 'https://www.scriptslug.com/script/belfast-2021',
 'https://www.scriptslug.com

In [97]:
#create empty list and append with parsed html        
script_slug_unremarkable_pdf_list= []
for url in script_slug_unremarkable_url_list_final:
  script_slug_unremarkable_pdf_list.append(get_script_slug_pdf(url))
print(script_slug_unremarkable_pdf_list)

Checking site status code for: https://www.scriptslug.com/script/bridge-of-spies-2015
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/script/chernobyl-104-the-happiness-of-all-mankind-2019
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/script/annabelle-2014
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/script/d2-the-mighty-ducks-1994
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/script/isle-of-dogs-2018
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/script/the-big-short-2015
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/script/eye-in-the-sky-2016
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/script/bean-1997
Response recieved: 200
Checking site status code for: https://www.scriptslug.com/script/childs-play-1988
Response recieved: 200
Checking site s

In [98]:
#generate single list from list-of-lists and then deduplicate urls
script_slug_unremarkable_pdf_list = (reduce(operator.concat, script_slug_unremarkable_pdf_list))
script_slug_unremarkable_pdf_list = (set(script_slug_unremarkable_pdf_list))
print('The size of the non-Oscar nominated document list will be',len(script_slug_unremarkable_pdf_list),'screenplays shown below.')
print(script_slug_unremarkable_pdf_list)

The size of the non-Oscar nominated document list will be 54 screenplays shown below.
{'https://www.scriptslug.com/assets/scripts/the-karate-kid-part-iii-1989.pdf', 'https://www.scriptslug.com/assets/scripts/trick-r-treat-2007.pdf', 'https://www.scriptslug.com/assets/scripts/belfast-2021.pdf', 'https://www.scriptslug.com/assets/scripts/king-of-the-hill-1012-24-hour-propane-people-2006.pdf', 'https://www.scriptslug.com/assets/scripts/air-force-one-1997.pdf', 'https://www.scriptslug.com/assets/scripts/touch-of-evil-1958.pdf', 'https://www.scriptslug.com/assets/scripts/blood-diamond-2006.pdf', 'https://www.scriptslug.com/assets/scripts/how-to-train-your-dragon-2010.pdf', 'https://www.scriptslug.com/assets/scripts/candyman-2021.pdf', 'https://www.scriptslug.com/assets/scripts/bean-1997.pdf', 'https://www.scriptslug.com/assets/scripts/cabin-in-the-woods-2012.pdf', 'https://www.scriptslug.com/assets/scripts/heaven-can-wait-1978.pdf', 'https://www.scriptslug.com/assets/scripts/the-thin-red-li

In [99]:
#downloading the non-Oscar nominated films
unremarkable_output_dir = 'C:/Users/U6068968/Downloads/Assessment3/unremarkable_pdf'

for url in script_slug_unremarkable_pdf_list:
    response = requests.get(url)
    if response.status_code == 200:
        file_path = os.path.join(unremarkable_output_dir, os.path.basename(url))
        with open(file_path, 'wb') as f:
            f.write(response.content)

In [105]:
import glob
from tika import parser 
from pandas import DataFrame

# What file extension to find, and where to look from
ext = "*.pdf"
PATH = "C:/Users/U6068968/Downloads/Assessment3/"

# Find all the files with that extension
files = []
for dirpath, dirnames, filenames in os.walk(PATH):
    files += glob.glob(os.path.join(dirpath, ext))

# Create a Pandas Dataframe to hold the filenames and the text
df = DataFrame(columns=("filename","text"))

# Process each file in turn, parsing with Tika and storing in the dataframe
for idx, filename in enumerate(files):
   data = parser.from_file(filename)
   text = data["content"]
   df.loc[idx] = [filename, text]

# For debugging, print what we found
print(df)

EmptyDataError: No columns to parse from file

In [ ]:
import nltk
nltk.download('punkt')
import pandas as pd
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import requests
!pip install -U LeXmo
from LeXmo import LeXmo

In [ ]:
#text pre-processing

#removing all punctuation
def removing_punctuation(string):
    try:
        string = string.str.replace(r'[^\w\s]+', '')
    except:
        pass
    return string

response_df_merged= response_df_merged.apply(removing_punctuation)

In [ ]:
#tokenization

response_df_merged=response_df_merged.fillna('')

for col in ['google_title', 'google_descripton', 'open_lib_title','open_lib_subtitle', 'open_lib_publisher', 
                 'subject_1','subject_2','subject_3','subject_4','open_lib_author_1','open_lib_author_2']:
    
    response_df_merged[col] = response_df_merged[col].apply(word_tokenize)

In [ ]:
#removing stopwords
stop = stopwords.words('english')

for col in ['google_title', 'google_descripton', 'open_lib_title','open_lib_subtitle', 'open_lib_publisher', 
                 'subject_1','subject_2','subject_3','subject_4','open_lib_author_1','open_lib_author_2']:

    response_df_merged[col]=response_df_merged[col].apply(lambda x: [item for item in x if item not in stop])

In [ ]:
#lemmatization 

lemmatizer = WordNetLemmatizer()
response_df_merged['google_descripton']=response_df_merged['google_descripton'].apply(lambda x:
                                                                                      [lemmatizer.lemmatize(word) for word in x])

In [81]:
#ensuring PEP8 compliance with nbQA


     ---------------------------------------- 61.9/61.9 kB ? eta 0:00:00
  Using cached mccabe-0.7.0-py2.py3-none-any.whl (7.3 kB)
     ---------------------------------------- 66.1/66.1 kB ? eta 0:00:00
     ---------------------------------------- 41.5/41.5 kB ? eta 0:00:00
  Attempting uninstall: pyflakes
    Found existing installation: pyflakes 2.4.0
    Uninstalling pyflakes-2.4.0:
      Successfully uninstalled pyflakes-2.4.0
  Attempting uninstall: pycodestyle
    Found existing installation: pycodestyle 2.8.0
    Uninstalling pycodestyle-2.8.0:
      Successfully uninstalled pycodestyle-2.8.0
  Attempting uninstall: mccabe
    Found existing installation: mccabe 0.6.1
    Uninstalling mccabe-0.6.1:
      Successfully uninstalled mccabe-0.6.1
  Attempting uninstall: flake8
    Found existing installation: flake8 4.0.1
    Uninstalling flake8-4.0.1:
      Successfully uninstalled flake8-4.0.1
